In [1]:
import random

import torch
from sklearn.datasets import fetch_covtype
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

random.seed(42)
torch.manual_seed(42)

In [13]:
class Model(torch.nn.Module):
    """
    This is our network
    """

    def __init__(self, p, input_dim, output_dim) -> None:
        super().__init__()

        ## TODO: Implement the model with activation functions and dropout. You
        ##       can use torch.nn.Sequential for your implementation.
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 30),
            torch.nn.ReLU(),
            Dropout(p),

            torch.nn.Linear(30, 30),
            torch.nn.ReLU(),
            Dropout(p),

            torch.nn.Linear(30, output_dim))
            
    def forward(self, x):
        return self.layers(x)

In [3]:
class Dropout(torch.nn.Module):
    """
    The dropout module required to implement.
    """

    def __init__(self, p) -> None:
        super().__init__()

        ## TODO: Implement the dropout module. You can use
        ##       torch.distributions.Bernoulli for your implementation.
        self.bernoulli = torch.distributions.Bernoulli(probs= (1 - p))
        self.scale = 1.0 / (1.0 - p) if p > 0 else 1.0
        self.p = p

    def forward(self, x):
        """
        Executes the dropout logic. Multiply some of the inputs
        with zero with p probability and scale with 1/(1-p) if
        in training mode.

        In eval mode don't do anything.
        """
        ## TODO: Implement the dropout logic.
        if self.training and self.p > 0.0:
            mask = self.bernoulli.sample(sample_shape=x.size()).to(x.device)
            return x * mask * self.scale
        else:
            return x

In [4]:

def prepare_dataloaders():
    """
    Load the data, preprocess it, and wrap it in torch dataloaders.
    return: train_loader, val_loader, test_loader
    """
    # Take the covtype dataset as it is reasonably large even though it is not
    # a binary task

    ## TODO: Load the covtype dataset from sklearn and split it into train,
    ##       val, and test split.
    data = fetch_covtype()
    x = data.data
    y = data.target

    # We pick the majority classes 1 and 2 as binary classification filter for
    # those instances
    x = x[(y == 1) | (y == 2)]
    y = y[(y == 1) | (y == 2)]

    # Normalize
    #
    # There is just one categorical variable in there (Cover_Type), we ignore
    # it here. You should properly encode it in practice (e.g. one-hot)!
    ## TODO: Normalize the data.
    scaler = StandardScaler()
    x = scaler.fit_transform(x)    
    # set classes to be in {0, 1}
    y[(y == 2)] = 0

    # split
    ## TODO: Split the data into train, val, and test split. Test size should
    ##       be 0.3, train size 0.35, and val size 0.35.
    x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.65, random_state=42)
    x_val, x_test, y_val, y_test = train_test_split(
        x_temp, y_temp, test_size=0.4615, random_state=42
    ) 
    ## TODO: Convert the data to torch tensors.
    x_train = torch.tensor(x_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    x_val = torch.tensor(x_val, dtype=torch.float32)
    y_val = torch.tensor(y_val, dtype=torch.long)
    x_test = torch.tensor(x_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.long)
    ## TODO: Dataloader for train, val, and test split with batch size 512.
    # Note: If your PC crashes / runs out of memory, try reducing the batch size
    train_dataset = TensorDataset(x_train, y_train)
    val_dataset = TensorDataset(x_val, y_val)
    test_dataset = TensorDataset(x_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

    return train_loader, val_loader, test_loader

In [15]:
def train_loop(train_loader, val_loader, lr, p, epochs=15):
    """
    The main train loop where we train the model and obtain the validation
    accuracy.

    Args:
        train_loader: torch dataloader for the train split
        val_loader: torch dataloader to evaluate the model
        lr: learning rate
        p: drop out probability
        epochs=15: number of epochs

    Returns:
        validation accuracy,
        trained model
    """
    tqdm.write(f"Running training with lr={lr}, p={p}")

    ## TODO: Initalize model, use Adam as optimizer and BCEWithLogitsLoss as
    ##       loss function.
    model = Model(p=p, input_dim=54, output_dim=1)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    # train
    model.train()
    for _ in tqdm(range(epochs), leave=False):
        accuracy = 0

        for x, y in tqdm(train_loader, leave=False):
            logits = model(x)

            ## TODO: Finish implementing the training loop.
            loss = loss_fn(logits.squeeze(), y.float())
            optimizer.zero_grad()  # Zero out gradients
            loss.backward()  # Backpropagation
            optimizer.step()  # Update model parameters
            ## Help for calculating some statistics
            y_hat = torch.sigmoid(logits) > .5
            accuracy += accuracy_score(y.numpy(), y_hat.numpy())

        accuracy /= len(train_loader)

        # Comment in the following line to see the train performance (take
        # numbers with grain of salt due to dropout)
        #
        #print(f"Epoch {e} train accuracy {accuracy:04f}")

    # val
    ## TODO: Implement the validation loop.
    accuracy = evaluate(model, val_loader)

    tqdm.write(f"Validation accuracy {accuracy:04f}")

    return accuracy, model

In [6]:
def evaluate(model, data_loader):
    """
    A simple function to evaluate the performance of a model.

    Args:
        model: torch.nn.Module to evaluate
        data_loader: the dataloader to evaluate on.

    Returns:
        The accuracy of the model on the given data.
    """
    model.eval()
    with torch.no_grad():
        accuracy = 0
        for x, y in data_loader:
            logits = model(x)
            y_hat = torch.sigmoid(logits) > 0.5
            accuracy += accuracy_score(y.numpy(), y_hat.numpy())

        accuracy /= len(data_loader)
        return accuracy


In [17]:
def main():
    """
    What we are gonna do for this exercise.
    """

    # The dataset is large enough to have a train, validation, and test split.
    # We used this to make the effects of dropout clearly visible.
    # If it is smaller use Cross-Validation as asked in the exercise sheet.
    # It is a little more to implement but should lead to similar insights.

    ## TODO: Dataloader for train, val, and test split.
    train_loader, val_loader, test_loader = prepare_dataloaders()
    # Report test performance.
    ## TODO: Train loop with hyperparameters lr=0.0001, p=0.4 and report test performance.
    lr = 0.0001
    p = 0.4
    epochs = 15  # Default epochs
    val_acc, trained_model = train_loop(train_loader, val_loader, lr, p, epochs)
    test_acc = evaluate(trained_model, test_loader)

    print(f"Test accuracy: {test_acc :.04f}")

In [18]:
if __name__ == "__main__":
    main()

Running training with lr=0.0001, p=0.4


 98%|█████████████████████████████████████████████████████████████████████████████▏ | 331/339 [00:02<00:00, 118.95it/s]


Validation accuracy 0.790472
Test accuracy: 0.7889
